In [ ]:
import os

import numpy as np
import xarray as xr

from pymedphys.deliverydata import DeliveryData
from pymedphys.logfile import delivery_data_from_logfile

In [ ]:
logfile_path = os.path.join(
    '..', '..',
    'tests', 'data', 'trf', 'integrity4',
    'INT4 18_12_12 08_28_23 Z [ICOM_ 7_03_2018 15_21_51] PF_G90..trf'
)
logfile_path

In [ ]:
delivery_data = delivery_data_from_logfile(logfile_path)

In [ ]:
# delivery_data